In [31]:
# All imports

%matplotlib inline

import cv2
import glob, os
import imageio
import matplotlib.pyplot as plt
import numpy
import sys

from keras import backend as K
from keras import regularizers
from keras.callbacks import EarlyStopping
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.models import Sequential

from skimage import feature
from sklearn.metrics import roc_curve, auc
from sklearn.svm import SVC

from keras.preprocessing.image import img_to_array, array_to_img
from keras.utils.vis_utils import model_to_dot

from IPython.display import SVG, display


In [55]:
# Common Constants
IMG_WIDTH = 32
IMG_HEIGHT = 32

USE_RAW_IMAGE_FEATURES = True
USE_CANNY_IMAGE_FEATURES = True

if K.image_data_format() == 'channels_first':
    INPUT_SHAPE = (3, IMG_WIDTH * 2, IMG_HEIGHT)
else:
    INPUT_SHAPE = (IMG_WIDTH * 2, IMG_HEIGHT, 3)

In [4]:
# All image processing and plot related

def plotImage(image):
    cv_rgb2 = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.imshow(cv_rgb2)
    plt.show()

def plotImageFromPath(imagePath):
    plotImage(cv2.imread(imagePath))

def shape_cannyEdgesOld(cv2Image, cannyImagePath):
    v = numpy.median(cv2Image)
    sigma = 0.0
    # apply automatic Canny edge detection using the computed median
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    #print "Lower", lower, "Upper", upper
    grayScaleImage = cv2.cvtColor(cv2Image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(grayScaleImage, 10, 255)
    cv2.imshow('Canny edges',edges)
    cv2.imwrite(cannyImagePath, edges)

def shape_cannyEdges(cv2Image, cannyImagePath):
    cv2Image[0,0] = (255, 0, 0)
    cv2Image[0,1] = (0, 255, 0)
    cv2Image[0,2] = (0, 0, 255)
    
    v = numpy.median(cv2Image)
    sigma = 0.0
    # apply automatic Canny edge detection using the computed median
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    #print "Lower", lower, "Upper", upper
    grayScaleImage = cv2.cvtColor(cv2Image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(grayScaleImage, 10, 255)
    cv2.imshow('Canny edges',edges)
    cv2.imwrite(cannyImagePath, edges)
    
def shape_hog(cv2Image, fileName): 
    grayScaleImage = cv2.cvtColor(cv2Image, cv2.COLOR_BGR2GRAY)
    fd, hog_image = feature.hog(grayScaleImage, orientations=8, pixels_per_cell=(16, 16),cells_per_block=(1, 1), visualise=True)
    
    cv2.imshow('HOG',hog_image)
    cv2.imwrite(fileName, hog_image)
    
def create_canny_edge_image(origImageDir, origImageName):
    
    origImagePath = origImageDir + '/' + origImageName
    print(origImagePath)
    
    cannyImageName = "canny_" + origImageName
    cannyImagePath = origImageDir + '/' + cannyImageName
    plainImage = cv2.imread(origImagePath)
    shape_cannyEdges(plainImage, cannyImagePath)
    
    return cannyImagePath
    
def create_canny_for_all_img(origImageDir):
    imgFiles = [filename for filename in os.listdir(origImageDir)
                if (not filename.startswith("canny") and filename.endswith(".jpg") )]
    for origImageName in imgFiles:
        cannyImageName = "canny_" + origImageName
        cannyImagePath = origImageDir + '/' + cannyImageName
        
        if not os.path.exists(cannyImagePath):
            create_canny_edge_image(origImageDir, origImageName)
            
def create_color_histogram (origImageDir, origImageName):
    origImagePath = origImageDir + '/' + origImageName
    plainImage = cv2.imread(origImagePath)
    
    hist = cv2.calcHist([plainImage], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    hist = hist.flatten()
    print (hist)

In [44]:
# All image feature verctor related

def create_features_for_all_raw_img(origImageDir, isSpam):
    imgFiles = [filename for filename in os.listdir(origImageDir)
                if ((not filename.startswith("canny")) and filename.endswith(".jpg") )]

    res = [create_feature_vector(origImageDir, origImageName, isSpam) for origImageName in imgFiles]
    return numpy.array(res)
            
def create_features_for_all_canny_img(origImageDir, isSpam):
    imgFiles = [filename for filename in os.listdir(origImageDir)
                if (filename.startswith("canny") and filename.endswith(".jpg") )]
    res = [create_feature_vector(origImageDir, origImageName, isSpam) for origImageName in imgFiles]
    return numpy.array(res)
    
def create_feature_vector(origImageDir, origImageName, isSpam, numRow=IMG_HEIGHT, numCol=IMG_WIDTH):
    origImagePath = origImageDir + '/' + origImageName
    plainImage = cv2.imread(origImagePath)
    resizedImage = cv2.resize(plainImage, (numRow, numCol))
    h = resizedImage.shape[0]
    w = resizedImage.shape[1]
    res = numpy.ndarray(shape=(3, h, w), dtype=float, order='F')
    # loop over the image, pixel by pixel
    for y in range(0, h):
        for x in range(0, w):
            # threshold the pixel
            color = resizedImage[y,x] 
            res[0, y, x] = color[0] / 255.0
            res[1, y, x] = color[1] / 255.0
            res[2, y, x] = color[2] / 255.0
    return numpy.append(res.flatten(), isSpam)

def crete_canny_and_get_features_with_labels(directory, label, use_raw_images, use_canny_images):
    if (not use_raw_images) and (not use_canny_images):
        raise Exception('At least use_raw_images or use_canny_images need to be set to true')
        
    create_canny_for_all_img(directory)
    data_raw = create_features_for_all_raw_img(directory, label)
    data_canny = create_features_for_all_canny_img(directory, label)
    
    if (use_raw_images and use_canny_images):
        return numpy.column_stack((data_raw[:,:-1], data_canny))
    elif use_raw_images:
        return data_raw
    else:
        return data_canny

def get_feature_with_labels(
    ham_dir="NaturalImages",
    spam_dir="SpamImages",
    show=False,
    use_raw_images=True,
    use_canny_images=True
):
    data_ham = crete_canny_and_get_features_with_labels(
        directory=ham_dir,
        label=0,
        use_raw_images=use_raw_images,
        use_canny_images=use_canny_images
    )
    data_spam = crete_canny_and_get_features_with_labels(
        directory=spam_dir,
        label=1,
        use_raw_images=use_raw_images,
        use_canny_images=use_canny_images
    )
    
    data = numpy.concatenate((data_ham, data_spam), axis=0)
    numpy.random.shuffle(data)

    if show:
        #numpy.set_printoptions(threshold=sys.maxsize)
        numpy.set_printoptions(threshold=10)
        print("data_ham, using directory:", ham_dir)
        print(data_ham)
        print("data_spam, using directory:", spam_dir)
        print(data_spam)
        print("data")
        print(data)

    return data

In [7]:
# Plot related
def draw_accuracy_and_loss(history):
    # summarize history for accuracy
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    
def draw_roc(y_predicted, y_actual):
    fpr, tpr, thresholds = roc_curve(y_predicted, y_actual)
    roc_auc = auc(fpr, tpr)
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=1, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right")
    plt.show()
    
def add_plot(plt, svm_res, color, label):
    
    fpr, tpr, thresholds = roc_curve(svm_res[0], svm_res[1])
    roc_auc = auc(fpr, tpr)
    
    plt.plot(fpr, tpr, color=color, lw=1, label=(label + ' ROC curve (area = %0.2f)') % roc_auc)
    
    
def draw_roc_for_all(svm_rbf_public, svm_linear_public, svm_rbf_cha1, svm_linear_cha1, svm_rbf_cha2, svm_linear_cha2):
    plt.figure()
    
    #fpr, tpr, thresholds = roc_curve(y_predicted, y_actual)
    #roc_auc = auc(fpr, tpr)
    
    #plt.plot(fpr, tpr, color='darkorange', lw=1, label='ROC curve (area = %0.2f)' % roc_auc)
    
    add_plot(plt, svm_rbf_public, 'darkorange', 'SVM-RBF-DATASET1')
    add_plot(plt, svm_linear_public, 'blue', 'SVM-LINEAR-DATASET1')
    add_plot(plt, svm_rbf_cha1, 'green', 'SVM-RBF-DATASET2')
    add_plot(plt, svm_linear_cha1, 'magenta', 'SVM-LINEAR-DATASET2')
    add_plot(plt, svm_rbf_cha2, 'pink', 'SVM-RBF-DATASET3')
    add_plot(plt, svm_linear_cha2, 'brown', 'SVM-LINEAR-DATASET3')
    
    plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right")
    plt.show()

def plot_keras_model(model, show_shapes=True, show_layer_names=True):
    model_svg = model_to_dot(model, show_shapes=show_shapes, show_layer_names=show_layer_names).create(prog='dot',format='svg')
    display(SVG(model_svg))

In [8]:
# Machine learning algorithm related

# Feed forward
def run_feed_forward(data, do_plot=False):
    
    INPUT_DIMENSION = len(data[0]) - 1
    LAYER_1_DIMENSION = 300
    LAYER_2_DIMENSION = 300
    OUTPUT_DIMENSION = 1
    EPOCH = 100
    
    train_len = int(len(data) * .7)
    data_train = data[0:train_len]
    data_test = data[train_len:]

    num_col = len(data_train[0]) - 1

    x_train = data_train[:, 0:num_col]
    y_train = data_train[:, num_col:].flatten()

    x_test = data_test[:, 0:num_col]
    y_test = data_test[:, num_col:].flatten()
    
    # Early stopping  
    early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='auto')

    model = Sequential()
    model.add(Dense(LAYER_1_DIMENSION,
                    input_dim=INPUT_DIMENSION,
                    activation='relu'))
    # With regularizers
    #model.add(Dense(LAYER_1_DIMENSION,
    #                input_dim=INPUT_DIMENSION,
    #                activation='relu',
    #                kernel_regularizer=regularizers.l2(0.01),
    #                activity_regularizer=regularizers.l1(0.01)))
    model.add(Dropout(0.5))
    model.add(Dense(LAYER_2_DIMENSION,
                    activation='relu'))
    # With regularizers
    #model.add(Dense(LAYER_2_DIMENSION,
    #                activation='relu',
    #                kernel_regularizer=regularizers.l2(0.01),
    #                activity_regularizer=regularizers.l1(0.01)))
    model.add(Dropout(0.5))
    model.add(Dense(OUTPUT_DIMENSION, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])
    
    history = model.fit(x_train,
                        y_train,
                        epochs=EPOCH,
                        batch_size=64,
                        validation_split=0.15,
                        callbacks=[early_stop])
    
    score = model.evaluate(x_test, y_test, batch_size=64)
    print (score)
    
    if do_plot:
        plot_keras_model(model)
        draw_accuracy_and_loss(history)

    return score

# CNN
def run_cnn(data, do_plot=False):
    EPOCH = 100
    
    train_len = int(len(data) * .7)
    data_train = data[0:train_len]
    data_test = data[train_len:]

    num_col = len(data_train[0]) - 1

    # Convert 1d array back to image for convolution
    x_train_1D = data_train[:, 0:num_col]
    x_train = numpy.array([z.reshape(INPUT_SHAPE) for z in x_train_1D])
    y_train = data_train[:, num_col:].flatten()

    # Convert 1d array back to image for convolution
    x_test_1D = data_test[:, 0:num_col]
    x_test = numpy.array([z.reshape(INPUT_SHAPE) for z in x_test_1D])
    y_test = data_test[:, num_col:].flatten()
    
    # Early stopping  
    early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='auto')

    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=INPUT_SHAPE))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    
    model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])

    history = model.fit(x_train,
                        y_train,
                        epochs=EPOCH,
                        batch_size=64,
                        validation_split=0.33,
                        callbacks=[early_stop])

    score = model.evaluate(x_test, y_test, batch_size=64)
    print (score)
    
    if do_plot:
        plot_keras_model(model)
        draw_accuracy_and_loss(history)
        
    return score

# Svm
def run_svm(data, do_plot=False):
    
    train_len = int(len(data) * .7)
    data_train = data[0:train_len]
    data_test = data[train_len:]
    
    num_col = len(data_train[0]) - 1
    
    x_train = data_train[:, 0:num_col]
    y_train = data_train[:, num_col:].flatten()
    
    x_test = data_test[:, 0:num_col]
    y_test = data_test[:, num_col:].flatten()
    
    clf = SVC(C=1.0, kernel='rbf')
    clf.fit(x_train, y_train)
    y_predicted = clf.predict(x_test)
    score = clf.score(x_test, y_test) 
    print (score)
    
    if do_plot:
        draw_roc(y_predicted, y_test)
    
    return score

def run_svm_all(data, kernel='rbf'):
    train_len = int(len(data) * .7)
    data_train = data[0:train_len]
    data_test = data[train_len:]
    
    num_col = len(data_train[0]) - 1
    
    x_train = data_train[:, 0:num_col]
    y_train = data_train[:, num_col:].flatten()
    
    x_test = data_test[:, 0:num_col]
    y_test = data_test[:, num_col:].flatten()
    
    clf = SVC(C=1.0, kernel=kernel)
    clf.fit(x_train, y_train)
    y_predicted = clf.predict(x_test)
    
    return (y_predicted, y_test)

In [9]:
# Draws ROC for all SVM
"""
data = get_feature_with_labels(ham_dir="NaturalImages", spam_dir="SpamImages")
svm_rbf_public = run_svm_all(data, 'rbf')
svm_linear_public = run_svm_all(data, 'linear')

data = get_feature_with_labels(ham_dir="ChallengeHam1", spam_dir="ChallengeSpam1")
svm_rbf_cha1 = run_svm_all(data, 'rbf')
svm_linear_cha1 = run_svm_all(data, 'linear')

data = get_feature_with_labels(ham_dir="ChallengeHam2", spam_dir="ChallengeSpam2")
svm_rbf_cha2 = run_svm_all(data, 'rbf')
svm_linear_cha2 = run_svm_all(data, 'linear')

draw_roc_for_all(svm_rbf_public,svm_linear_public, svm_rbf_cha1,svm_linear_cha1, svm_rbf_cha2, svm_linear_cha2)
"""

'data = get_feature_with_labels(nonSpamDir="NaturalImages", spamDir="SpamImages")\nsvm_rbf_public = run_svm_all(data, \'rbf\')\nsvm_linear_public = run_svm_all(data, \'linear\')\n\ndata = get_feature_with_labels(nonSpamDir="ChallengeHam1", spamDir="ChallengeSpam1")\nsvm_rbf_cha1 = run_svm_all(data, \'rbf\')\nsvm_linear_cha1 = run_svm_all(data, \'linear\')\n\ndata = get_feature_with_labels(nonSpamDir="ChallengeHam2", spamDir="ChallengeSpam2")\nsvm_rbf_cha2 = run_svm_all(data, \'rbf\')\nsvm_linear_cha2 = run_svm_all(data, \'linear\')'

In [62]:
# For testing, uncomment and save data to reuse

data_public = get_feature_with_labels(
    ham_dir="MergedHam",
    spam_dir="MergedSpam",
    show=True,
    use_raw_images=USE_RAW_IMAGE_FEATURES,
    use_canny_images=USE_CANNY_IMAGE_FEATURES
)

#data = get_feature_with_labels(ham_dir="NaturalImages", spam_dir="SpamImages")

#data = get_feature_with_labels(ham_dir="ChallengeHam1", spam_dir="ChallengeSpam1")
#data = get_feature_with_labels(ham_dir="ChallengeHam2", spam_dir="ChallengeSpam2")
#data = get_feature_with_labels(ham_dir="MergedHam", spam_dir="MergedSpam")

MergedHam/hm1_zzz_10963_03507d6116_m.jpg
MergedHam/hm0_zzz_402090311_484cf5a5b2_t.jpg
MergedHam/hm0_zzz_90140_1e60d55f1f_m.jpg
MergedHam/hm0_zzz_30431_61bf54ef82_m.jpg
MergedHam/hm0_zzz_412528990_180ebd2c22_m.jpg
MergedHam/hm2_zzz_04167_dda98f63b5_m.jpg
MergedHam/hm2_zzz_413068575_f9e7067ffc_m.jpg
MergedHam/hm1_zzz_53382_3837f85a64_m.jpg
MergedHam/hm0_zzz_407644832_a3cd8708f9_m.jpg
MergedHam/hm0_zzz_402058348_91b74202a1_t.jpg
MergedHam/hm2_zzz_86725_1afc38079c_m.jpg
MergedHam/hm1_zzz_11105_9ef55f8725_m.jpg
MergedHam/hm0_zzz_49468_b286bb7676_m.jpg
MergedHam/hm1_zzz_408253921_5be1294e1a_m.jpg
MergedHam/hm2_zzz_78466_3f01d36308_m.jpg
MergedHam/hm0_zzz_59779_46b2f54dfa_m.jpg
MergedHam/hm1_zzz_00243_a8054e0b34_m.jpg
MergedHam/hm0_zzz_406886587_fb8a220ea7_m.jpg
MergedHam/hm0_zzz_54374_7642e2dea4_m.jpg
MergedHam/hm0_zzz_413210045_110327a32f_m.jpg
MergedHam/hm0_zzz_86412_f908d52027_m.jpg
MergedHam/hm0_zzz_410528404_8ba305ba90_m.jpg
MergedHam/hm0_zzz_scan03.jpg
MergedHam/hm1_zzz_412750287_716d7

MergedHam/hm0_zzz_402097266_61c6d28259_t.jpg
MergedHam/hm2_zzz_411138040_17bf08af0b_m.jpg
MergedHam/hm2_zzz_408715494_ddb88bb972_m.jpg
MergedHam/hm1_zzz_414747562_825d4ad42f_m.jpg
MergedHam/hm1_zzz_41898_20a8f2cc35_m.jpg
MergedHam/hm0_zzz_412738506_adc05bebf4_m.jpg
MergedHam/hm2_zzz_12886_31c61461af_m.jpg
MergedHam/hm1_zzz_scan04.jpg
MergedHam/hm2_zzz_414489291_659ef1df8f_m.jpg
MergedHam/hm0_zzz_92158_3fd28fa7ab_m.jpg
MergedHam/hm1_zzz_scan10.jpg
MergedHam/hm0_zzz_02866_cc49ace670_m.jpg
MergedHam/hm1_zzz_67473_27af1b9e54_m.jpg
MergedHam/hm2_zzz_401895241_fb23965818_t.jpg
MergedHam/hm2_zzz_91249_d20706fcbe_m.jpg
MergedHam/hm1_zzz_411969298_1de83882d5_m.jpg
MergedHam/hm2_zzz_21545_e53bfb983b_m.jpg
MergedHam/hm2_zzz_91064_01f964cdc0_m.jpg
MergedHam/hm0_zzz_test_40.jpg
MergedHam/hm0_zzz_412944137_74217eda56_m.jpg
MergedHam/hm2_zzz_48379_d240e8742c_m.jpg
MergedHam/hm0_zzz_65842_42fd2c24c7_m.jpg
MergedHam/hm2_zzz_43918_5e7a59d5f7_m.jpg
MergedHam/hm1_zzz_42499_f7277b30c1_m.jpg
MergedHam/hm1_z

MergedHam/hm2_zzz_408747260_4b4fe05df7_m.jpg
MergedHam/hm0_zzz_78695_be4aff0e7f_m.jpg
MergedHam/hm1_zzz_413026314_c8855d7883_m.jpg
MergedHam/hm0_zzz_01736_2f9f491a0b_m.jpg
MergedHam/hm2_zzz_414931919_07925b3bb6_m.jpg
MergedHam/hm2_zzz_40760_05ba3fe2e9_m.jpg
MergedHam/hm2_zzz_7093_f476297f45_m.jpg
MergedHam/hm1_zzz_407631468_b7274ddb61_m.jpg
MergedHam/hm2_zzz_36716_f6e6d438ca_m.jpg
MergedHam/hm2_zzz_33693_6f7fe56119_m.jpg
MergedHam/hm1_zzz_49044_2ccb6a2b15_m.jpg
MergedHam/hm0_zzz_21934_0c586b512a_m.jpg
MergedHam/hm1_zzz_05924_a4ae64d5f9_m.jpg
MergedHam/hm1_zzz_402080780_213976b903_t.jpg
MergedHam/hm2_zzz_413113710_7efff04e8e_m.jpg
MergedHam/hm0_zzz_63089_bff2c57e18_m.jpg
MergedHam/hm1_zzz_test_3.jpg
MergedHam/hm2_zzz_402088256_7185e5e16f_t.jpg
MergedHam/hm1_zzz_13581_64b6a41674_m.jpg
MergedHam/hm0_zzz_56335_fc20437fe2_m.jpg
MergedHam/hm0_zzz_412793614_0735f22ebd_m.jpg
MergedHam/hm2_zzz_67379_c2849e6d50_m.jpg
MergedHam/hm2_zzz_407674643_2072c9add9_m.jpg
MergedHam/hm0_zzz_63789_88a3a6a977

MergedHam/hm1_zzz_23689_aa96d2d719_m.jpg
MergedHam/hm2_zzz_68253_e3b1c92344_m.jpg
MergedHam/hm0_zzz_9814_3b88fd33e3_m.jpg
MergedHam/hm0_zzz_412160635_b570b143a1_m.jpg
MergedHam/hm2_zzz_412725173_b0c0e61ce4_m.jpg
MergedHam/hm1_zzz_55166_6699d2afcc_m.jpg
MergedHam/hm2_zzz_414426117_94d357ef53_m.jpg
MergedHam/hm1_zzz_72951_b648ae89d9_m.jpg
MergedHam/hm1_zzz_408731392_a81727a379_m.jpg
MergedHam/hm0_zzz_90569_137a88ecb2_m.jpg
MergedHam/hm0_zzz_413053384_40d5426bc1_m.jpg
MergedHam/hm1_zzz_23807_f3cea68c48_m.jpg
MergedHam/hm2_zzz_78528_7a3b56da50_m.jpg
MergedHam/hm2_zzz_43818_3fc7a0cb6d_m.jpg
MergedHam/hm1_zzz_412546717_088b305caf_m.jpg
MergedHam/hm2_zzz_test_14.jpg
MergedHam/hm1_zzz_21791_eaa61aa008_m.jpg
MergedHam/hm0_zzz_00724_d0ff7beb6c_m.jpg
MergedHam/hm0_zzz_73091_6ae879d7b8_m.jpg
MergedHam/hm2_zzz_417227878_60cd8277a1_m.jpg
MergedHam/hm1_zzz_10516_65e8559783_m.jpg
MergedHam/hm1_zzz_413461827_b9234bb18b_m.jpg
MergedHam/hm1_zzz_412723604_0787152aca_m.jpg
MergedHam/hm2_zzz_test_28.jpg
Mer

MergedHam/hm0_zzz_60648_776f8bdcf9_m.jpg
MergedHam/hm2_zzz_24793_ce10b53001_m.jpg
MergedHam/hm2_zzz_69273_e616977961_m.jpg
MergedHam/hm2_zzz_71905_c4cced8b02_m.jpg
MergedHam/hm1_zzz_14608_26ce2c6756_m.jpg
MergedHam/hm2_zzz_410803131_7a6d8c3e9a_m.jpg
MergedHam/hm0_zzz_409899893_e457ac3242_m.jpg
MergedHam/hm1_zzz_412622901_dca244ea1b_m.jpg
MergedHam/hm0_zzz_412333512_18aa502dbf_m.jpg
MergedHam/hm1_zzz_04529_4dd918ea7c_m.jpg
MergedHam/hm2_zzz_scan05.jpg
MergedHam/hm2_zzz_30558_d9b6ea669c_m.jpg
MergedHam/hm2_zzz_98815_06cae62cc3_m.jpg
MergedHam/hm1_zzz_31873_ff98ca02cf_m.jpg
MergedHam/hm1_zzz_412643423_6eca6d3975_m.jpg
MergedHam/hm2_zzz_34935_e557ee838b_m.jpg
MergedHam/hm2_zzz_scan11.jpg
MergedHam/hm2_zzz_00797_3369ff7b8a_m.jpg
MergedHam/hm2_zzz_412670069_cad21faf9f_m.jpg
MergedHam/hm1_zzz_26089_e9fc132e60_m.jpg
MergedHam/hm2_zzz_65288_fcbef362f9_m.jpg
MergedHam/hm2_zzz_22411_d1fdb98d38_m.jpg
MergedHam/hm2_zzz_408914630_f74f15ea4c_m.jpg
MergedHam/hm2_zzz_68609_b092825c4b_m.jpg
MergedHam/hm

...

MergedSpam/sp0_autobiography.jpg
MergedSpam/sp2_582.jpg
MergedSpam/sp1_441.jpg
MergedSpam/sp2_596.jpg
MergedSpam/sp1_455.jpg
MergedSpam/sp0_Lists.jpg
MergedSpam/sp0_cream-puff.jpg
MergedSpam/sp1_333.jpg
MergedSpam/sp1_469.jpg
MergedSpam/sp0_lmsqbakj.jpg
MergedSpam/sp0_anair.jpg
MergedSpam/sp1_680.jpg
MergedSpam/sp2_743.jpg
MergedSpam/sp0_mitigating.jpg
MergedSpam/sp1_694.jpg
MergedSpam/sp2_757.jpg
MergedSpam/sp0_esophagus.jpg
MergedSpam/sp1_643.jpg
MergedSpam/sp2_780.jpg
MergedSpam/sp1_125.jpg
MergedSpam/sp1_131.jpg
MergedSpam/sp0_satire.jpg
MergedSpam/sp1_657.jpg
MergedSpam/sp2_794.jpg
MergedSpam/sp1_119.jpg
MergedSpam/sp0_this.jpg
MergedSpam/sp0_rightangle.jpg
MergedSpam/sp1_118.jpg
MergedSpam/sp1_130.jpg
MergedSpam/sp2_795.jpg
MergedSpam/sp1_656.jpg
MergedSpam/sp2_781.jpg
MergedSpam/sp1_642.jpg
MergedSpam/sp1_124.jpg
MergedSpam/sp0_batt.jpg
MergedSpam/sp0_antipodean.jpg
MergedSpam/sp2_756.jpg
MergedSpam/sp1_695.jpg
MergedSpam/sp2_742.jpg
MergedSpam/sp1_681.jpg
MergedSpam/sp0_IndexVi

MergedSpam/sp2_584.jpg
MergedSpam/sp2_779.jpg
MergedSpam/sp2_751.jpg
MergedSpam/sp1_692.jpg
MergedSpam/sp0_T0o1N4kQVX.jpg
MergedSpam/sp2_745.jpg
MergedSpam/sp1_686.jpg
MergedSpam/sp0_2Trb4akgIN.jpg
MergedSpam/sp0_21HlnMZdf4.jpg
MergedSpam/sp1_679.jpg
MergedSpam/sp0_SMTcf2oCXB.jpg
MergedSpam/sp1_137.jpg
MergedSpam/sp0_masseuse.jpg
MergedSpam/sp2_792.jpg
MergedSpam/sp1_651.jpg
MergedSpam/sp2_786.jpg
MergedSpam/sp1_645.jpg
MergedSpam/sp1_123.jpg
MergedSpam/sp1_644.jpg
MergedSpam/sp2_787.jpg
MergedSpam/sp1_122.jpg
MergedSpam/sp1_136.jpg
MergedSpam/sp1_650.jpg
MergedSpam/sp2_793.jpg
MergedSpam/sp1_678.jpg
MergedSpam/sp0_expulsion.jpg
MergedSpam/sp0_can-they.jpg
MergedSpam/sp0_3dZWE0FjvG.jpg
MergedSpam/sp1_687.jpg
MergedSpam/sp2_744.jpg
MergedSpam/sp1_693.jpg
MergedSpam/sp2_750.jpg
MergedSpam/sp2_778.jpg
MergedSpam/sp1_320.jpg
MergedSpam/sp0_3e5VQKygiz.jpg
MergedSpam/sp2_585.jpg
MergedSpam/sp1_446.jpg
MergedSpam/sp2_591.jpg
MergedSpam/sp1_452.jpg
MergedSpam/sp1_334.jpg
MergedSpam/sp0_zgilgga

MergedSpam/sp0_Desk.jpg
MergedSpam/sp0_isgBFl655K.jpg
MergedSpam/sp1_688.jpg
MergedSpam/sp0_summer-school.jpg
MergedSpam/sp0_stately.jpg
MergedSpam/sp2_777.jpg
MergedSpam/sp2_763.jpg
MergedSpam/sp1_449.jpg
MergedSpam/sp0_abstinent.jpg
MergedSpam/sp0_yeroakobf.jpg
MergedSpam/sp0_Fora.jpg
MergedSpam/sp1_475.jpg
MergedSpam/sp0_9LPSEZpPVv.jpg
MergedSpam/sp1_313.jpg
MergedSpam/sp1_307.jpg
MergedSpam/sp0_cBfUekgbne.jpg
MergedSpam/sp0_GMd88mBPvA.jpg
MergedSpam/sp1_461.jpg
MergedSpam/sp0_jwwx.jpg
MergedSpam/sp2_549.jpg
MergedSpam/sp0_uksxhp_7.jpg
MergedSpam/sp0_botanist.jpg
MergedSpam/sp2_575.jpg
MergedSpam/sp2_213.jpg
MergedSpam/sp2_207.jpg
MergedSpam/sp2_561.jpg
MergedSpam/sp0_Be2qNQ6B52.jpg
MergedSpam/sp2_398.jpg
MergedSpam/sp0_image_spam20.jpg
MergedSpam/sp1_529.jpg
MergedSpam/sp1_501.jpg
MergedSpam/sp2_53.jpg
MergedSpam/sp1_267.jpg
MergedSpam/sp0_R.jpg
MergedSpam/sp1_273.jpg
MergedSpam/sp2_47.jpg
MergedSpam/sp1_515.jpg
MergedSpam/sp1_298.jpg
MergedSpam/sp2_429.jpg
MergedSpam/sp2_401.jpg
M

MergedSpam/sp2_215.jpg
MergedSpam/sp1_498.jpg
MergedSpam/sp2_229.jpg
MergedSpam/sp1_275.jpg
MergedSpam/sp0_0uS3tts9xP.jpg
MergedSpam/sp2_41.jpg
MergedSpam/sp1_513.jpg
MergedSpam/sp1_507.jpg
MergedSpam/sp0_CS5X375.jpg
MergedSpam/sp2_55.jpg
MergedSpam/sp1_261.jpg
MergedSpam/sp1_249.jpg
MergedSpam/sp0_sdvhwueviqa.jpg
MergedSpam/sp0_ipfyads.jpg
MergedSpam/sp2_69.jpg
MergedSpam/sp2_82.jpg
MergedSpam/sp2_375.jpg
MergedSpam/sp2_413.jpg
MergedSpam/sp2_407.jpg
MergedSpam/sp2_361.jpg
MergedSpam/sp2_96.jpg
MergedSpam/sp2_349.jpg
MergedSpam/sp1_711.jpg
MergedSpam/sp2_7.jpg
MergedSpam/sp0_adpcvwp.jpg
MergedSpam/sp1_38.jpg
MergedSpam/sp1_705.jpg
MergedSpam/sp1_10.jpg
MergedSpam/sp2_188.jpg
MergedSpam/sp1_739.jpg
MergedSpam/sp2_611.jpg
MergedSpam/sp2_177.jpg
MergedSpam/sp2_163.jpg
MergedSpam/sp2_605.jpg
MergedSpam/sp0_sewcrxncl.jpg
MergedSpam/sp2_639.jpg
MergedSpam/sp0_kxo_1.jpg
MergedSpam/sp2_104.jpg
MergedSpam/sp2_662.jpg
MergedSpam/sp0_psfv_0.jpg
MergedSpam/sp1_88.jpg
MergedSpam/sp2_676.jpg
Merged

MergedSpam/sp2_658.jpg
MergedSpam/sp0_pgm4osj3c.jpg
MergedSpam/sp2_670.jpg
MergedSpam/sp2_116.jpg
MergedSpam/sp0_twU9DBGXBV.jpg
MergedSpam/sp0_Sn82YN8LuZ.jpg
MergedSpam/sp2_102.jpg
MergedSpam/sp0_diversity.jpg
MergedSpam/sp2_664.jpg
MergedSpam/sp1_71.jpg
MergedSpam/sp0_metaphorical.jpg
MergedSpam/sp1_758.jpg
MergedSpam/sp1_65.jpg
MergedSpam/sp1_770.jpg
MergedSpam/sp0_image102.jpg
MergedSpam/sp0_bludgeon.jpg
MergedSpam/sp0_unable.jpg
MergedSpam/sp1_764.jpg
MergedSpam/sp1_59.jpg
MergedSpam/sp2_328.jpg
MergedSpam/sp1_599.jpg
MergedSpam/sp0_signatory.jpg
MergedSpam/sp0_plant.jpg
MergedSpam/sp0_CIMG3917.jpg
MergedSpam/sp2_314.jpg
MergedSpam/sp2_472.jpg
MergedSpam/sp2_466.jpg
MergedSpam/sp2_300.jpg
MergedSpam/sp1_228.jpg
MergedSpam/sp0_ppWSPEJC1C.jpg
MergedSpam/sp2_499.jpg
MergedSpam/sp1_214.jpg
MergedSpam/sp2_20.jpg
MergedSpam/sp0_tenderize.jpg
MergedSpam/sp1_572.jpg
MergedSpam/sp1_566.jpg
MergedSpam/sp2_34.jpg
MergedSpam/sp1_200.jpg
MergedSpam/sp0_fGhqtV5GQq.jpg
MergedSpam/sp0_berserk.jpg


In [54]:
# Main

# Returns an array conaining 10, 20, .. 90, 100, 200, ..., len(data)
def find_iterations(data):
    total = len(data)
    start = 10
    
    print (total)
    iters = []
    while start < total:
        iters.append(start)
        if start < 100:
            start += 10
        else:
            start += 100
    iters.append(total)
    return iters

def main():
    find_iterations(data_public)
    
def main2():
    # This fixes how many rows to consider for each run
    rowIter = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200, 300,
               400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500, 1530]
    rowIter = [10]
    # How many iterations
    numIter = 1
    
    # Holds results
    accuracies = dict()
    
    print (data)
    
    for numRowsToConsider in rowIter:
        sum = 0
        #data = get_feature_with_labels(ham_dir="ChallengeHam1", spam_dir="ChallengeSpam1")
        #data = get_feature_with_labels(ham_dir="ChallengeHam2", spam_dir="ChallengeSpam2")
        for i in range(0, numIter):
            sum += run_svm(data, True)
            #sum += run_feed_forward(data, True)[1]
            #sum += run_cnn(data, True)[1]
        sum /= numIter
        print (sum)
        accuracies[numRowsToConsider] = sum
        
    print (accuracies)

main()

1727
[10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700]


In [ ]:
# This section and rest are just tests
def extract_features(imagePath, vector_size=32):
    image = imageio.imread(imagePath)
    alg = cv2.KAZE_create()
    kps = alg.detect(image)
    kps = sorted(kps, key=lambda x: -x.response)[:vector_size]
    print (len(kps))
    kps, dsc = alg.compute(image, kps)
    print (len(dsc))
    print (len(dsc[0]))
    dsc = dsc.flatten()
    print (len(dsc))
    needed_size = (vector_size * 32)
    if dsc.size < needed_size:
        dsc = numpy.concatenate([dsc, np.zeros(needed_size - dsc.size)])
    return dsc

def main2():
    #features = create_feature_vector("SpamImages", "fire.jpg")
    #print len(features)
    #print features
    #image = cv2.imread("SpamImages/fire.jpg")
    #plotImage(image)
    #image2 = cv2.resize(image, (600, 600))
    #plotImage(image2)
    
    
    """data = get_feature_with_labels()
    data"""
    
    #print (len(data[0]))
    
    

    #create_feature_vector_for_canny("NaturalImages", origImageName)
    
    #plainImageFileName = "SpamImages/boots1.jpg"
    """plainImage =  cv2.imread(plainImageFileName)
    shape_hog(plainImage, "HOG-Ham.png")
    
    #plt.imshow(plainImage, cmap = 'gray', interpolation = 'bicubic')
    #plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
    #plt.show()
    cv_rgb = cv2.cvtColor(plainImage, cv2.COLOR_BGR2RGB)
    plt.imshow(cv_rgb)
    plt.show()
    
    plainImage2 =  cv2.imread("HOG-Ham.png")

    print "hi"
    
    cv_rgb2 = cv2.cvtColor(plainImage2, cv2.COLOR_BGR2RGB)
    plt.imshow(cv_rgb2)
    plt.show()

    shape_cannyEdges(plainImage)
    cv_rgb2 = cv2.cvtColor(cv2.imread("Cannyedges.jpg"), cv2.COLOR_BGR2RGB)
    plt.imshow(cv_rgb2)
    plt.show()"""
    
    #x = create_feature_vector_for_canny("SpamImages", "fire.jpg", 3.0)
    numpy.set_printoptions(threshold=100)
    #print x
    
    x = numpy.ndarray((10, 3))
    print(x)
    train_len = int(len(x) * .7)
    #print x[0:train_len]
    #print "sep"
    #print x[train_len:]
    
    num_col = len(x[0]) - 1
    
    print(x[:,0:num_col])
    
    print(x[:, num_col:].flatten())